In [2]:
import json
import nltk
import re
import string
from nltk.corpus import stopwords 
import demoji
from nltk.chunk import conlltags2tree, tree2conlltags
import spacy
from imdb import IMDb, IMDbError
import itertools
import demoji
stop_words = set(stopwords.words('english')) 

In [3]:
OFFICIAL_AWARDS_1315 = [
    'cecil b. demille award', 
    'best motion picture - drama', 
    'best performance by an actress in a motion picture - drama', 
    'best performance by an actor in a motion picture - drama', 
    'best motion picture - comedy or musical', 
    'best performance by an actress in a motion picture - comedy or musical', 
    'best performance by an actor in a motion picture - comedy or musical',
    'best animated feature film', 'best foreign language film', 
    'best performance by an actress in a supporting role in a motion picture', 
    'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 
    'best screenplay - motion picture', 
    'best original score - motion picture', 
    'best original song - motion picture', 
    'best television series - drama', 
    'best performance by an actress in a television series - drama', 
    'best performance by an actor in a television series - drama', 
    'best television series - comedy or musical', 
    'best performance by an actress in a television series - comedy or musical', 
    'best performance by an actor in a television series - comedy or musical', 
    'best mini-series or motion picture made for television', 
    'best performance by an actress in a mini-series or motion picture made for television', 
    'best performance by an actor in a mini-series or motion picture made for television', 
    'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 
    'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']
OFFICIAL_AWARDS_1819 = [
    'best motion picture - drama', 
    'best motion picture - musical or comedy', 
    'best performance by an actress in a motion picture - drama', 
    'best performance by an actor in a motion picture - drama', 
    'best performance by an actress in a motion picture - musical or comedy', 
    'best performance by an actor in a motion picture - musical or comedy', 
    'best performance by an actress in a supporting role in any motion picture', 
    'best performance by an actor in a supporting role in any motion picture', 
    'best director - motion picture', 
    'best screenplay - motion picture', 
    'best motion picture - animated', 
    'best motion picture - foreign language', 
    'best original score - motion picture', 
    'best original song - motion picture', 
    'best television series - drama', 
    'best television series - musical or comedy', 
    'best television limited series or motion picture made for television', 
    'best performance by an actress in a limited series or a motion picture made for television', 
    'best performance by an actor in a limited series or a motion picture made for television', 
    'best performance by an actress in a television series - drama', 
    'best performance by an actor in a television series - drama', 
    'best performance by an actress in a television series - musical or comedy', 
    'best performance by an actor in a television series - musical or comedy', 
    'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 
    'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 
    'cecil b. demille award']

In [3]:
flag = 2013
if flag == 2020:
    with open('gg2020.json', encoding='utf8') as json_file:
        data = [json.loads(line) for line in json_file]
else:
    data = json.load(open('gg2013.json'))
    official_award = OFFICIAL_AWARDS_1315

NameError: name 'OFFICIAL_AWARDS_1315' is not defined

In [17]:
award_dict = {}
nominee = {}
for award_name in official_award:
    award_dict[award_name] = {}
    reduced_award_name = award_name.replace("award", "")
    reduced_award_name = reduced_award_name.replace(" - ", " ")
    reduced_award_name = reduced_award_name.replace(",", "")
    reduced_award_name = reduced_award_name.replace(" b. ", " ")
    reduced_award_name = reduced_award_name.replace("performance", "")
    reduced_award_name = reduced_award_name.replace(" by ", " ")
    reduced_award_name = reduced_award_name.replace(" an " , " ")
    reduced_award_name = reduced_award_name.replace(" in a " , " ")
    reduced_award_name = reduced_award_name.replace(" or " , " ")
    reduced_award_name = reduced_award_name.replace(" film" , "")
    reduced_award_name = reduced_award_name.replace(" role " , " ")
    reduced_award_name = reduced_award_name.replace(" series " , " ")
    reduced_award_name = reduced_award_name.replace(" for " , " ")
    reduced_award_name = reduced_award_name.replace("best " , "")
    reduced_award_name = reduced_award_name.replace("mini-series", "mini")
    award_dict[award_name]['search_word'] = reduced_award_name.split()
    ambiguity_award_name = reduced_award_name.replace(" motion picture", "")
    ambiguity_award_name = reduced_award_name.replace(" television", "")
    award_dict[award_name]['ambiguity'] = ambiguity_award_name.split()
    award_dict[award_name]['prohibit'] = []
    if "television" in reduced_award_name:
        award_dict[award_name]['prohibit'].append("motion")
        award_dict[award_name]['prohibit'].append("picture")
    elif "motion picture" in reduced_award_name:
        award_dict[award_name]['prohibit'].append("television")
        award_dict[award_name]['prohibit'].append("series")   
    if "act" not in reduced_award_name:
        award_dict[award_name]['prohibit'].append("actor")
        award_dict[award_name]['prohibit'].append("actress")
    if "act" in reduced_award_name and "supporting" not in reduced_award_name:
        award_dict[award_name]['prohibit'].append("supporting")
    nominee[award_name] = []


In [4]:
new_str = string.punctuation
new_str += '“'
#new_str += '–'
def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in new_str:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [5]:
clean_data = []
for tweet in data:
    tweet_text = tweet['text']
    no_http = re.sub('http://\S+|https://\S+', '', tweet_text)
    remove_tag = strip_all_entities(no_http)
    new_item = re.sub(r'–','-',remove_tag)
    new_item = re.sub(r'[Tt].[Vv].','television',new_item)
    new_item = re.sub(r'[Tt][Vv]','television',new_item) 
    new_item = re.sub(r'RT ','',new_item) 
    clean_data.append(new_item.lower())
clean_data.sort()
clean_data = list(set(clean_data))

In [7]:
pattern_nominee = re.compile(r'[Nn]omin[ae]')
nominee_data = []
for tweet in clean_data:
    if len(re.findall(pattern_nominee, tweet)) > 0:
        nominee_data.append(tweet)

In [22]:
#start search
for award in award_dict:
    print(award)
    print(award_dict[award]['search_word'])
    print("----------------------")
    for tweet in nominee_data:
        # search perfect match 
        if all(word in tweet for word in award_dict[award]['search_word']) and 'oscar' not in tweet and 'disney' not in tweet and 'win' not in tweet:
            print(tweet)
        # search ambiguity
        elif all(word in tweet for word in award_dict[award]['ambiguity']) and not any(word in tweet for word in award_dict[award]['prohibit'])  and 'oscar' not in tweet and 'disney' not in tweet and 'win' not in tweet:
            print(tweet)
    print("----------------------")
        

cecil b. demille award
['cecil', 'demille']
----------------------
----------------------
best motion picture - drama
['motion', 'picture', 'drama']
----------------------
here it is best motion picture drama argo and to think ben affleck was not nominated hmmmm
pretty sweet they had president clinton introduce the movie lincoln nominated for best motion picture drama
bill clinton introduced for the best motion picture drama nomination at the
----------------------
best performance by an actress in a motion picture - drama
['actress', 'motion', 'picture', 'drama']
----------------------
----------------------
best performance by an actor in a motion picture - drama
['actor', 'motion', 'picture', 'drama']
----------------------
----------------------
best motion picture - comedy or musical
['motion', 'picture', 'comedy', 'musical']
----------------------
it s weird that they put comedy and musical in the same category best motion picture considering les mis is nominated
----------------

In [9]:
for key in award_dict:
    print(award_dict[key])

['cecil', 'demille']
['best', 'motion', 'picture', 'drama']
['best', 'actress', 'motion', 'picture', 'drama']
['best', 'actor', 'motion', 'picture', 'drama']
['best', 'motion', 'picture', 'comedy', 'musical']
['best', 'actress', 'motion', 'picture', 'comedy', 'musical']
['best', 'actor', 'motion', 'picture', 'comedy', 'musical']
['best', 'animated', 'feature']
['best', 'foreign', 'language']
['best', 'actress', 'supporting', 'motion', 'picture']
['best', 'actor', 'supporting', 'motion', 'picture']
['best', 'director', 'motion', 'picture']
['best', 'screenplay', 'motion', 'picture']
['best', 'original', 'score', 'motion', 'picture']
['best', 'original', 'song', 'motion', 'picture']
['best', 'television', 'drama']
['best', 'actress', 'television', 'drama']
['best', 'actor', 'television', 'drama']
['best', 'television', 'comedy', 'musical']
['best', 'actress', 'television', 'comedy', 'musical']
['best', 'actor', 'television', 'comedy', 'musical']
['best', 'mini', 'motion', 'picture', 'mad

In [24]:
imdb_base = IMDb()

In [1]:
check_human_type = ["actor", "actress", "demille", "director"]

In [15]:
for award in award_dict:
    for tweet in clean_data:
        #if all(word in tweet for word in award_dict[award]):
        if 'demille' in tweet:
            if any(word in tweet for word in nominee_words):
                print(tweet)
    break

In [15]:
pat = re.compile('.*(hop(ed|ing|e|es))\s(@)?(\w+)\s(w(o|i)(n|ns|nning)).*', re.IGNORECASE)

In [16]:
for tweet in clean_data:
    if len(re.findall(pat, tweet)) > 0:
        print(tweet)

i hope adele wins skyfall is the greatest song
something about bon jovi and christopher walken in the same movie awesome but i hope adele wins
hi benedict you are adorable and i want to touch your face hope you win
i hope she wins just showed jennifer she s wearing a jacket she must be extra cold with that flu
poor leo i really hope he wins something someday
i really really really hope adele wins a globe tonight she looks gorg in
let s all listen to safe and sound and hope it wins
thebrunetteone ben affleck you are hot i hope you win i hope kathryn bigelow wins too but i hope you win
i think i died when the les mis trailer was played i really hope it wins
is fucking adorable great look for her good job hope you win tonight
congratulations mr affleck for best director in a motion picture drama i hope this win makes up for the oscar
jessica lange you lovely woman i hope you win
en route to golden globes hope i win
i love hugh so much but i thought bradley would win cuz he‘s great in silv

In [11]:
for tweet in clean_data:
    if "nicole kidman" in tweet:
        print(tweet)

favorite looks by far kate hudson emily blunt and nicole kidman
we spy nicole kidman and already did you spot them
so far nicole kidman has my vote for best dress
nicole kidman flaunts an amazing figure in our very own alexander mcqueen
nicole kidman in alexander mcqueen 2013 ohh i love nicole
nicole kidman and keith urban arrive at the
kind of upset nicole kidman didn t win would have liked to have seen her pee on the stage
nicole kidman misses out on best performance by an actress in a mini series or a motion picture made for television to julianne moore
my picks for best dressed of the night michelle dockery kerry washington amp nicole kidman
nicole kidman at the 2013 golden globes
no words nicole kidman in alexander mcqueen
its gossip night on golden globes dresses up 2nd nicole kidman like or trash i vote like
here is keith urban amp nicole kidman in alexander mcqueen
izleyemiyorum çünkü television bozuldu ama red carpet olayını yine nicole kidman götürmüştür diyorum
nicole kidman

In [13]:
pattern_miss = re.compile(r'miss')
miss_data = []
for tweet in clean_data:
    if len(re.findall(pattern_miss, tweet)) > 0:
        miss_data.append(tweet)
        print(tweet)

waaaaaaa missing the golden globes lool
yo le quiero dar el premio miss simpatía a ang lee porque cada vez que lo enfocaron se reía
i m glad the golden globes is starting over again i missed half of it now i get to rewatch jodie foster s beautiful speech
am i the only one watching revenge right now the golden globes are great but i m not missing my show i ll flip over after loo
if by any chance you missed the golden globes this evening all you need to know is this took place
aww thank you i miss my tay tay in da winnnn who definitely made an appearance on the tonight
apparently missed the best of all time
hearing kiefer talk about a vice presidential candidate makes me miss 24
adele won omg my baby we re only missing the oscar omfg so proud
i miss ricky gervais presenting the
missing the because you ve been at work since 1
missed the golden globes and now because people in my dorm are live streaming django to the communal television honestly a little pissed
everyone s golden globes and

mt brooklyn s got a miss america amp is the scenic backdrop 4 winner girls rent 2 go up 50 tomorrow borough wide
missing tonight s episode but congrats to maggie smith s win
i refused to get my flu shot today just to watch the tonight sorry for not wanting to miss who s wearing the best dresses
la única película en ganar el óscar sin ser nominado su director fue driving miss daisy en el 89 sera argo la segunda
dang just realized i missed the
is it me or does it seem like francesca eastwood does not feel like being at the or miss golden globe at all
i m gonna miss the golden globes xc noooooooo d
missing the golden globes amp ouat 😱😭
amy poehler not coming out as hillary while bill clinton was on stage biggest missed moment at the
nooooooo i hate golden globes taylor missed best original song in golden globe the award went to adele with skyfall
panicking because something happened at the and you can t see it what is going on with jodie foster what am i missing
all this talk about jodi f

In [6]:
#check these
for tweet in clean_data:
    r = re.findall("(.*) (?:(?:not win)|(?:is a lock)|(?:should win)|(?:should not win)|(?:better not win)|(?:didn\'t win)|(?:doesn\'t win)|(?:deserves to win)|(?:deserved to win)|(?:better win)|(?:is nominated for)|(?:was nominated for)|(?:will win)|(?:should've won)|(?:could win)|(?:has to win)|(?:is going to win)|(?:is gonna win)|(?:win ))", tweet, re.IGNORECASE)
    print(r)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['if damien lewis does']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['i wanted the hot girl from rust and bone” to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congrats to all the noms and winners tonight highlight for me was jessica chastain s']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['seriously you give day lewis 30 seconds the guy everyone thought was going to']
[]
[]
['did deserve to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['and the award for best acting like she didn t know she was going to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['the golden globes is always promising so happy the nominees i wanted to']
[]
[]
[]
[]
['i m just glad didn t win still sad that suddenly didn t']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['really happy taylor swift didn t']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['of course james dolan s table doesn t']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['live updates argo les miserables']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['bradley cooper for the']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['i bet bill clinton is going to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['rust and bone is gona']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['bradcoop didn t']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['live updates argo les miserables']
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['and for once something that taylor swift didnt']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['i need to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['how the hell did benedict cumberbatch']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['homeland wins best television drama really wanted breaking bad to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ben affleck has to work on his acceptance speeches some time until he will']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['this is so mean but dear god please don t let anne hathaway']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['django should ve']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['why didn t benedict cumberbatch']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

...

[]
[]
[]
[]
[]
[]
['hey when you know someone is gonna']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['guys what is that diet pepsi commercial nominated for because i don t think it']
[]
['really wanted bradley cooper to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congrats to damian lewis damian on his']
[]
[]
['waiting for jessica chastain to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['wow ben affleck shock']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congratulations on your']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['finally an award her legion of 14 yr old fans can t']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congrats to on their first']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['the first to make a movie mocking tebow will definitely']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congratulations to adele for her golden globes']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['he did']
[]
[]
[]
[]
[]
[]
['here we go tweet along w us during the golden globes tweet for a chance to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['leonardo dicaprio should just']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['of course skyfall']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['jlawrence for the']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['golden globes 2013 life of pi and skyfall']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[]
[]
[]
[]
[]
[]
[]
[]
[]
['am i correct that damien lewis s']
[]
['i m very conflicted over this best actress in a comedy']
[]
[]
[]
[]
[]
[]
[]
['sofia should just']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['would have liked to see wes']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['congrats on the']
[]
[]
['amy poehler didn t even have to']
[]
['the golden globes should have an hnic award jamie foxx should be the only one nominated and']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['homeland downton abbey']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['who would have predicted christoph waltz to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['assuming anne hathaway is going to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['bet u r right rt@jimmyktown re golden globes put homeland table closer to the stage they are always going to']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['silver lining playbook should']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['hugh jackman wins actor in a musical comedy for i think that mean the film will']
[]
[]
[]
['the cuts to are kinda funny she s not up for awards but she looks disappointed she s']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]